<a href="https://colab.research.google.com/github/chakma21/BTP/blob/main/Exploringlib/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uploading files

In [23]:
# Step 1: Upload the file
from google.colab import files
uploaded = files.upload()

# Get the filename
filename = list(uploaded.keys())[0]

# Read and join all paragraphs as one big text
with open(filename, 'r', encoding='utf-8') as f:
    content = f.read()


lines = content.strip().splitlines()
filtered_lines = [line.strip() for line in lines if not line.strip().startswith('-') and line.strip() != '']
combined_text = ' '.join(filtered_lines)


# Split on paragraph breaks (double newline), then join with space
# paragraphs = content.strip().split('\n\n')
# combined_text = ' '.join(paragraph.strip().replace('\n', ' ') for paragraph in paragraphs)

print(f" Combined {len(paragraphs)} paragraphs into one block of {len(combined_text.split())} words.")


Saving catalan_independence to catalan_independence (1)
 Combined 621 paragraphs into one block of 23683 words.


Installing requirements (you only need to run it one time when running the code for the first time)

In [4]:
!pip install rake-nltk yake
# !pip uninstall -y transformers
!pip install transformers --upgrade

!pip install --upgrade huggingface_hub
!pip install --upgrade keybert sentence-transformers

import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')


Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.52.4-py3-none-any.whl (10.5 MB)


ERROR: Operation cancelled by user
^C


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

import and extractors defining

In [24]:
# RAKE
from rake_nltk import Rake

# YAKE
import yake

# KeyBERT
from keybert import KeyBERT

# Initialize models
rake = Rake()
kw_extractor_yake = yake.KeywordExtractor(lan="en", top=10)
model_keybert = KeyBERT()


functions for each extractor

In [25]:
def extract_keywords_rake(text, top_n=100):
    rake.extract_keywords_from_text(text)
    return rake.get_ranked_phrases_with_scores()[:top_n]

def extract_keywords_yake(text):
    return kw_extractor_yake.extract_keywords(text)

def extract_keywords_keybert(text):
    return model_keybert.extract_keywords(
        text,
        top_n=100,
        stop_words='english',
        keyphrase_ngram_range=(1, 3)
    )


Calling the functions and printing

In [50]:
print("\n==============================")
print(" Extracting keywords from entire article collection\n")

print("---  RAKE Keywords ---")
for score, kw in extract_keywords_rake(combined_text):
    print(f"{kw} ({score:.2f})")

print("\n---  YAKE Keywords ---")
for kw, score in extract_keywords_yake(combined_text):
    print(f"{kw} ({score:.6f})")

print("\n---  KeyBERT Keywords ---")
for kw, score in extract_keywords_keybert(combined_text):
    print(f"{kw} ({score:.4f})")



 Extracting keywords from entire article collection

---  RAKE Keywords ---
la independencia de cataluña se desploma tras las elecciones 21d secesión 1o artículo 155 referéndum dui https :// (291.48)
mobile world congress 🇪🇸 cacerolaso por la llegada del rey de españa (108.24)
spain explainedimage copyright afp image caption carles puigdemont rallies separatists via videolink (79.50)
“ completely unexplainable .” ciutadans urges maintaining article 155 unionist ciutadans (74.27)
women “ unexplainable ,” says catecp catalunya en comú podem (65.54)
falling apart ,” kepler cheuvreux analyst adrian zunzunegui said (63.13)
5umkhjudhf — puerto de la cruz (@ puertocruz05 (59.76)
ministers “ effective immediately .” esquerra republicana celebrates news toni comín (54.56)
swiss modelimage copyright epa image caption mr puigdemont fled spain (54.50)
basque nationalist mp izaskun bilbao demanded urgent eu action (53.99)
judge pablo llarena said 25 catalan separatist leaders -- including puigdemo

In [46]:
rake_keywords = extract_keywords_rake(combined_text)  # (score, keyword)
yake_keywords = extract_keywords_yake(combined_text)  # (keyword, score)
keybert_keywords = extract_keywords_keybert(combined_text)  # (keyword, score)
print("Succesfully stored")


Succesfully stored


CLASSIFY WHETHER TARGET OR NOT

In [53]:
def classify_keywords(keyword_list, method_keywords, threshold, method_name):
    if method_name == "RAKE":
        # RAKE gives (score, phrase), so we reverse it
        kw_dict = {kw.lower(): score for score, kw in method_keywords}
    else:
        kw_dict = {kw.lower(): score for kw, score in method_keywords}

    results = {}

    for keyword in keyword_list:
        keyword_lower = keyword.lower()
        found = False

        for phrase, score in kw_dict.items():
            if keyword_lower in phrase:
                # print(f"[DEBUG] {keyword_lower} matched phrase '{phrase}' with score {score}")
                if method_name in ["RAKE", "KeyBERT"]:
                    if score >= threshold:
                        found = True
                        break
                else:  # YAKE: lower is better
                    if round(score, 6) < round(threshold, 6):
                        found = True
                        break

        results[keyword] = found

    return results


For every new article make sure to change the keyword to test and the threshold values.

In [55]:
keywords_to_test = ["Catalan independence"]
raketest=["susana vera “ falling apart ” puigdemont appeared","mobile world congress","former parliament speaker carme forcadell"]
yaketest=["Spain","Catalan independence"]
keyberttest=["catalonia want independence","regional election catalonia","catalonians voted","spain called elections"]

rake_threshold = 30
yake_threshold = 0.000277  # e.g. raw score, lower means more important
keybert_threshold = 0.63

rakewords= classify_keywords(raketest, rake_keywords, rake_threshold, "RAKE")
yakewords= classify_keywords(yaketest, yake_keywords, yake_threshold, "YAKE")
keybertwords= classify_keywords(keyberttest, keybert_keywords, keybert_threshold, "KeyBERT")

print("Rake classification")
for word, is_target in rakewords.items():
    print(f"{word} → {is_target}")
print("\n")

print("Yake classification")
for word, is_target in yakewords.items():
    print(f"{word} → {is_target}")
print("\n")

print("Keybert classification")
for word, is_target in keybertwords.items():
    print(f"{word} → {is_target}")


Rake classification
susana vera “ falling apart ” puigdemont appeared → True
mobile world congress → True
former parliament speaker carme forcadell → False


Yake classification
Spain → False
Catalan independence → True


Keybert classification
catalonia want independence → True
regional election catalonia → True
catalonians voted → False
spain called elections → False
